In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
%matplotlib inline

In [2]:
data = pd.read_csv('Amazon.csv')
data = data[0:1000]
data.shape

(1000, 13)

In [3]:
# data.head(5)

In [4]:
# define y
y = data.iloc[:, 12].values
y.shape

(1000,)

In [5]:
#bag of words on text
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()

type(data.Text)
textArray = data.Text.as_matrix()
type(textArray)
features = count.fit_transform(textArray)
XText = features.toarray()
XText.shape

(1000, 6831)

In [6]:
#bag of words on summary
# creating newSummary with "no summary given" where summary does not exist.
# data['SummaryFill'] = np.where(pd.isnull(data['Summary']) == True, 'no summary given', data['Summary'])
data['Summary'].fillna('null', inplace=True)
print(type(data.Summary))
summaryArray = data.Summary.as_matrix()
type(summaryArray)
features = count.fit_transform(summaryArray)
XSummary = features.toarray()
XSummary.shape

<class 'pandas.core.series.Series'>


(1000, 1309)

In [7]:
# features from Amazon.csv to add to feature set
data['reviewLen'] = data['Text'].str.len()
data['summaryLen'] = data['Summary'].str.len()
data['usernameWords'] = data ['ProfileName'].str.split().str.len()

XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XsummaryLen = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XusernameWords =data.iloc[:, 15].values.reshape(data.shape[0], 1)

In [8]:
Xtoadd = np.concatenate((XText,XSummary,XScore,XreviewLen,XsummaryLen,XusernameWords), axis=1)

print(Xtoadd)
print(Xtoadd.shape)

[[   0.    0.    0. ...,  206.   16.    3.]
 [   0.    0.    0. ...,  125.    5.    2.]
 [   0.    0.    0. ...,  678.   13.    1.]
 ..., 
 [   0.    0.    0. ...,  364.   24.    2.]
 [   0.    0.    0. ...,  307.    7.    1.]
 [   0.    0.    0. ...,  259.   39.    4.]]
(1000, 8144)


In [9]:
# report on training and test sets
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [10]:
# vectorize Bag of Words from Summary text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
XhashText = hv.transform(data.Text)
XhashSummary = hv.transform(data.Summary)

In [11]:
#Fix missing values in summary
# data["Summary"].fillna(data["Summary"].mean(), inplace=True)
# data["Summary"].fillna('pad')
# data["Summary"].fillna('pad')
# data.Summary.fillna(0)
# data['newSummary'] = np.where(pd.isnull(data['Summary']) == True, 'no summary given', data['Summary'])
# Y = hv.transform(data.Summary)

# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# imp.fit(hv.transform(data.Summary))

In [12]:
# convert to CSR
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
# concatenate to BOW CSR matrix
Xfinal = hstack([Xtoadd, XtoaddSparse])
X = csr_matrix(Xfinal)
type(X)

scipy.sparse.csr.csr_matrix

In [13]:
# size of feature set
print(Xtoadd.shape)
# define y = "Is it helpful?"
y = data.iloc[:, 12].values
y.shape

(1000, 8144)


(1000,)

In [14]:
Xfinal.shape
# # create training and test sets
# X_train = Xfinal[0:5]
# X_test = Xfinal[5:8]
# y_train = y[0:5]
# y_test = y[5:8]
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

(1000, 16288)

In [15]:
# define y
y = data.iloc[:, 12].values
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         Xfinal, y, test_size=0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(700, 16288)
(700,)
(300, 16288)
(300,)


In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1.0, random_state=0)
y_pred = lr.fit(X_train, y_train).predict(X_train)
y_pred_test = lr.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [17]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [18]:
print_results()

Error rate on training set: 


NameError: name 'y_pred' is not defined

In [ ]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_train)
y_pred_test = svm.predict(X_test)
print_results()